In [1]:
import os
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import cv2

import tensorflow
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.layers import Flatten, Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image 
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import ResNet50

In [ ]:
img_labels = pd.read_csv("data/driver_imgs_list.csv")

train_image = []
image_label = []


for i in range(10):
    imgs = os.listdir("data/imgs/train/c"+str(i))
    for j in range(len(imgs)):
        img_name = "data/imgs/train/c"+str(i)+"/"+imgs[j]
        img = cv2.imread(img_name)
        img = img[50:,120:-50]
        img = cv2.resize(img,(224,224))
        label = i
        driver = img_labels[img_labels['img'] == imgs[j]]['subject'].values[0]
        train_image.append([img,label,driver])
        image_label.append(i)

In [ ]:
driv_selected = ['p050', 'p015', 'p022', 'p056']

In [ ]:
X_train= []
y_train = []
X_test = []
y_test = []
D_train = []
D_test = []

for features,labels,drivers in train_image:
    if drivers in driv_selected:
        X_test.append(features)
        y_test.append(labels)
        D_test.append(drivers)
    
    else:
        X_train.append(features)
        y_train.append(labels)
        D_train.append(drivers)

print (len(X_train),len(X_test))
print (len(y_train),len(y_test))


In [ ]:
X_train = np.array(X_train).reshape(-1,224,224,3)
X_test = np.array(X_test).reshape(-1,224,224,3)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )
data_generator = datagen.flow(X_train, y_train, batch_size = 128)

In [ ]:
input_shape = Input(shape = (224, 224, 3), name = 'Image_input')
model = ResNet50(weights='imagenet', include_top=False, input_tensor = input_shape)

model.trainable = False

output_model = model(input_shape)
x = GlobalAveragePooling2D()(output_model)
x = BatchNormalization()(x)
x = Dropout(0.1)(x) 
x = Dense(1024,activation='relu')(x) 
x = Dropout(0.5)(x)
x = Dense(512,activation='relu')(x) 
x = Dense(10, activation='softmax', name='predictions')(x)

resnet = Model(input_shape, x)

resnet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

resnet.summary()

# resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
# output = resnet.layers[-1].output
# output = layers.Flatten()(output)


# resnet = Model(resnet.input, output)


# for layer in resnet.layers:
#     layer.trainable = False

# model = Sequential()

# model.add(resnet)

# model.add(Dense(512, activation='relu', input_dim=(224,224,3)))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(10, activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# model.summary()

In [ ]:
resnet.fit(data_generator, validation_data=(X_test, y_test), batch_size = 128, epochs=10)

In [ ]:
accNT = mmodel_history.history['accuracy']
val_accNT = mmodel_history.history['val_accuracy']
lossNT = mmodel_history.history['loss']
val_lossNT = mmodel_history.history['val_loss']
epochsNT = range(len(accNT))

plt.plot(epochsNT, accNT, 'r', label='Training accuracy')
plt.plot(epochsNT, val_accNT, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

plt.plot(epochsNT, lossNT, 'r', label='Training loss')
plt.plot(epochsNT, val_lossNT, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()